In [9]:
import pandas as pd
import numpy as np

def tchebycheff_score(fx, z_star, weights):
    """
    Tchebycheff scalarization function.
    
    Parameters:
        fx (np.ndarray): Objective matrix of shape (n_samples, n_objectives)
        z_star (np.ndarray): Ideal point (minimum for each objective)
        weights (np.ndarray): Weight vector for objectives
        
    Returns:
        np.ndarray: Scalarized scores (one per row)
    """
    diff = np.abs(fx - z_star)
    weighted = weights * diff
    return np.max(weighted, axis=1)

# === File Settings ===
# input_file = "rank_SA_NSGA.xlsx"       # Replace with actual filename
input_file = "mobo_pareto.csv"       # Replace with actual filename
output_file = "ranked_MOBO.xlsx"
sheet_name = "Sheet1"

# === Step 1: Load Data ===
# df = pd.read_excel(input_file, sheet_name=sheet_name)
df = pd.read_csv(input_file)

# === Step 2: Transform objectives to minimization form ===
# Accuracy becomes (1 - Accuracy), Size_MB and FPR are already minimized
df['1-Accuracy'] = 1.0 - df['Accuracy']
objective_matrix = df[['1-Accuracy', 'Size_MB', 'FPR']].values

# === Step 3: Compute Ideal Point (z*) ===
z_star = objective_matrix.min(axis=0)

# === Step 4: Define Equal Weights ===
weights = np.array([1.0, 1.0, 1.0])
weights = weights / weights.sum()  # Normalize to [1/3, 1/3, 1/3]

# === Step 5: Calculate Tchebycheff Score ===
df['Tchebycheff Score'] = tchebycheff_score(objective_matrix, z_star, weights)

# === Step 6: Rank Based on Tchebycheff Score ===
df['Rank'] = df['Tchebycheff Score'].rank(method='min', ascending=True).astype(int)

# === Step 7: Save Output ===
df_sorted = df.sort_values(by='Rank')  # Optional: sorted by best rank first
df_sorted.to_excel(output_file, index=False)

print(f"✅ Done. Ranked solutions saved to: {output_file}")


✅ Done. Ranked solutions saved to: ranked_MOBO.xlsx
